<h1>Setting up Environment </h1>

In [1]:
#Parameters
nb_actions = 5
i_width = 227
i_height = 227

In [5]:
from rl.core import Env
import numpy as np
import sys
import scipy.misc
sys.path.append('/home/arusia/fastai/airsim/AirSim/PythonClient')
from AirSimClient import *
class simulation(Env):
    import time
    reward_range = (-np.inf, np.inf)
    observation_space = None
    solar_coordinates = [3800,180,100]
    old_dist=0
    scale_reward_1=0.3
    state=None
    
    def __init__(self):
        
        # connect to the AirSim simulator
        self.client = MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        self.client.takeoff()
        self.state=self.client.getMultirotorState()
        
        
    def get_image(self):
        image_response = self.client.simGetImages([ImageRequest(0, AirSimImageType.Scene, False, False)])[0]
        image1d = np.fromstring(image_response.image_data_uint8, dtype=np.uint8)
        image_rgba = image1d.reshape(image_response.height, image_response.width, 4)
        image_rgba = image_rgba[:,:,:3]
        img=scipy.misc.imresize(image_rgba,[i_height, i_width])
        return img
    
    def get_distance(self):
        alt=self.state.gps_location.altitude
        lat=self.state.gps_location.latitude
        lon=self.state.gps_location.longitude
        coord=np.array([lat,lon,alt])
        return np.sqrt(np.linalg.norm(coord-self.solar_coordinates))
    
    def get_reward(self):
        self.new_dist=self.get_distance()
        if(self.state.collision.has_collided==True):
            reward=-1
            return
        if(self.new_dist<10):
            reward=1
            return
        reward = self.scale_reward_1/(-self.new_dist+self.old_dist)
        self.old_dist=self.new_dist
        return reward
        
    def step(self, action):
        #perform action
        #find the direction to move to
        normpara=np.linalg.norm(action[0:3])
        direc = action[0:3]/normpara
        
        #find the velocity in each direction
        velocity = direc * action[3]
        
        self.client.moveByVelocity(velocity[0], velocity[1], velocity[2], action[4])
        time.sleep(action[4])
        
        #Get observations
        self.state=self.client.getMultirotorState()
        observation=self.get_image()
        reward = self.get_reward()
        done=False
        if(self.new_dist<10):
            done=True
        info=None
        return observation,reward,done,info
    
    def reset(self):
        self.client.reset()
        client = MultirotorClient()
        client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        client.takeoff()
        self.old_dist=self.get_distance()
        observation=self.get_image()
        self.state=self.client.getMultirotorState()
        return observation
    
    def delete(self):
        self.client.reset()
        self.client.enableApiControl(False)

        
    def a_space(self):
        def sample(self,seed=None):
            direction=-1+np.random.random(1)*2
            speed_time=np.random.random(1)
            return [direction[0],direction[1],direction[2],speed_time[0],speed_time[1]]
        def contains(self,x):
            if(max(x[0:3])>1 or min(x[0:3])<-1):
                return False
            
            if(max(x[0:3])>1 or min(x[0:3])<0):
                return False
            return True
        
    action_space = a_space

<h1> Getting vanilla SqueezeNet to be used as the driver </h1>

In [6]:
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image

In [9]:
from keras import layers
from keras.layers import Dense,Flatten,Input,Concatenate
from keras.models import Model

def get_actor():
    model = SqueezeNet()
    hid=Dense(512,activation='relu')(model.layers[-2].output)
    x=Dense(1, activation='tanh')(hid)
    y=Dense(1, activation='tanh')(hid)
    z=Dense(1, activation='tanh')(hid)
    v=Dense(1, activation='sigmoid')(hid)
    t=Dense(1, activation='sigmoid')(hid)
    out=Concatenate()([x,y,z,v,t])
    Driver = Model(inputs=model.layers[0].output, outputs=out)
    return Driver

def get_critic():
    model = SqueezeNet()
    
    features=model.layers[-2].output
    action=Input(shape=(nb_actions,),name="input_action")
    conc=Concatenate()([features,action])
    hid=Dense(512,activation='relu')(conc)
    output_layer=Dense(1, activation='sigmoid')(hid)
    Driver = Model(inputs=[model.layers[0].output,action], outputs=output_layer)
    return Driver

<h1>Writing a reinforcement learning routine</h1>

In [10]:
from rl.random import OrnsteinUhlenbeckProcess
class random_proc(OrnsteinUhlenbeckProcess):
    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        x[self.size-1]=abs(x[self.size-1])
        self.x_prev = x
        self.n_steps += 1
        return x
        

In [11]:
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
from keras.optimizers import Adam

from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess


ENV_NAME='airsim'
env=simulation()
# Get actor and critic
actor=get_actor()
critic=get_critic()

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=100000, window_length=1)

random_process = random_proc(size=nb_actions, theta=.15, mu=0., sigma=.3)

agent = DDPGAgent(nb_actions=nb_actions, actor=actor, critic=critic, critic_action_input=critic.input[1],
                  memory=memory, nb_steps_warmup_critic=100, nb_steps_warmup_actor=100,
                  random_process=random_process, gamma=.97, target_model_update=1e-3)
agent.compile(Adam(lr=.001, clipnorm=1.), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
agent.fit(env, nb_steps=50000, visualize=False, verbose=1, nb_max_episode_steps=200)

# After training is done, we save the final weights.
agent.save_weights('ddpg_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
agent.test(env, nb_episodes=5, visualize=True, nb_max_episode_steps=200)

Waiting for connection: 
Training for 50000 steps ...
Waiting for connection: 
Interval 1 (0 steps performed)


/home/arusia/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/home/arusia/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


ValueError: Error when checking : expected input_1 to have 4 dimensions, but got array with shape (1, 1, 227, 227, 3)